# Limpieza y preprocesamiento de datos

## ¿Qué es limpieza de datos?

La _limpieza de datos_ es el proceso de identificar y corregir (o eliminar) errores y inconsistencias en los datos para mejorar su calidad.

Incluye tareas como:

* Manejo de valores faltantes
* Remover valores duplicados
* Corregir valores erróneos o inconsistentes
* Estandarizar formatos
* Manejo de valores atípicos (outliers)

## ¿Qué es preprocesamiento de datos?

El _preprocesamiento de datos_ es el proceso de transformar y preparar los datos brutos para que sean adecuados para el análisis o modelado.

Incluye tareas como:

* Escalado y normalización de datos
* Codificación de variables categóricas
* Reducción de dimensionalidad
* Separar data en train/test

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = sns.load_dataset("titanic")
df.head()

In [ ]:
df.info()

In [ ]:
df.pclass.unique() #class

##### 🔹 Conocer el tipo de variable y valores

| variable      | tipo de variable                     | descripción breve                                      | ejemplos de valores |
|---------------|--------------------------------------|--------------------------------------------------------|---------------------|
| survived      | cualitativa dicotómica (binaria)     | indica si sobrevivió (1) o no (0)                      | `0`, `1` |
| pclass        | cualitativa ordinal                  | clase del pasajero (nivel socioeconómico)              | `1`, `2`, `3` |
| sex           | cualitativa nominal                  | sexo biológico del pasajero                            | `"male"`, `"female"` |
| age           | cuantitativa continua (ratio)        | edad en años                                           | `22.0`, `35.0`, `nan` |
| sibsp         | cuantitativa discreta                | número de hermanos/esposos a bordo                     | `0`, `1`, `3` |
| parch         | cuantitativa discreta                | número de padres/hijos a bordo                         | `0`, `1`, `2` |
| fare          | cuantitativa continua (ratio)        | tarifa pagada                                          | `7.25`, `71.28`, `512.33` |
| embarked      | cualitativa nominal                  | puerto de embarque (c, q, s)                           | `"s"`, `"c"`, `"q"` |
| class         | cualitativa ordinal (categoría)      | clase en texto                                         | `"first"`, `"second"`, `"third"` |
| who           | cualitativa nominal                  | tipo de persona                                        | `"man"`, `"woman"`, `"child"` |
| adult_male    | cualitativa dicotómica (binaria)     | indica si es hombre adulto                             | `True`, `False` |
| deck          | cualitativa nominal (muchos nulos)   | letra de la cubierta                                   | `"c"`, `"e"`, `nan` |
| embark_town   | cualitativa nominal                  | ciudad de embarque                                     | `"southampton"`, `"cherbourg"`, `"queenstown"` |
| alive         | cualitativa dicotómica (binaria)     | estado de supervivencia en texto                       | `"yes"`, `"no"` |
| alone         | cualitativa dicotómica (binaria)     | indica si viajaba solo                                 | `True`, `False` |



## Comencemos por la limpieza de datos

##### 🔹Valores faltantes

In [ ]:
# Valores faltantes
df.isna().sum()

Cuando tenemos valores faltantes en un dataset, podemos manejarlos de varias maneras:

* Eliminar filas o columnas con valores faltantes
* Imputar valores faltantes con la media, mediana o moda
* Usar modelos predictivos para estimar los valores faltantes

**OJO**: **Nunca debemos imputar valores faltantes con la media, mediana o moda sin antes analizar si la variable tiene una distribución normal o sesgada. También es importante considerar que este paso requiere su debida atención, ya que puede introducir sesgos en el análisis.**

In [ ]:
# Ejemplo: Imputar variable "Age" condicionada a "Pclass" y "Sex"

age_median = df.groupby(['pclass', 'sex'])['age'].median()
age_median

In [ ]:
def impute_age(row):
    if pd.isna(row["age"]):
        return age_median.loc[row["pclass"], row["sex"]]
    return row["age"]

In [ ]:
df["age"] = df.apply(impute_age, axis=1)

In [ ]:
df.isna().sum()

_¿Qué piensas del siguiente ejemplo? ¿Lo imputarías de esta manera?_

In [ ]:
# Imputar por moda
df["embarked"].fillna(df["embarked"].mode()[0], inplace=True)

##### 🔹 Valores duplicados

In [ ]:
# Valores duplicados
df.duplicated().sum()

Cuando trabajamos con `.duplicate()` podemos encontrar filas duplicadas en un DataFrame.

```python
   Nombre  Edad
0     Ana    25
1    Luis    30
2     Ana    25
3  Carlos    40

0    False
1    False
2     True   <- duplicado de la fila 0
3    False
dtype: bool

1   <- hay un duplicado
```

A menos que podamos justificar la presencia de filas duplicadas, generalmente es una buena práctica eliminarlas para evitar sesgos en el análisis.

In [ ]:
# Eliminar duplicados
df = df.drop_duplicates()

##### 🔹 Estandarizar formatos

In [ ]:
df["sex"] = df["sex"].str.lower()  
df["embarked"] = df["embarked"].str.upper()

##### 🔹Tratar _outliers_

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12,5))

# Boxplot
ax[0].boxplot(df["fare"], vert=False)
ax[0].set_title("Fare - Antes (boxplot)")

# Histograma
ax[1].hist(df["fare"], bins=30, color="skyblue", edgecolor="black")
ax[1].set_title("Fare - Antes (histograma)")

plt.show()

In [ ]:
# Calcular los límites con IQR
Q1 = df["fare"].quantile(0.25)   # primer cuartil (25%)
Q3 = df["fare"].quantile(0.75)   # tercer cuartil (75%)
IQR = Q3 - Q1                    # rango intercuartílico

lower = Q1 - 1.5 * IQR            # límite inferior
upper = Q3 + 1.5 * IQR            # límite superior

print(f"Límite inferior: {lower:.2f}")
print(f"Límite superior: {upper:.2f}")

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))

ax.hist(df["fare"], bins=30, color="skyblue", edgecolor="black")
ax.axvline(lower, color="green", linestyle="--", linewidth=2, label=f"Límite inferior = {lower:.2f}")
ax.axvline(upper, color="red", linestyle="--", linewidth=2, label=f"Límite superior = {upper:.2f}")

ax.set_title("Distribución de fare con límites IQR")
ax.set_xlabel("Fare")
ax.set_ylabel("Frecuencia")
ax.legend()

plt.show()

In [ ]:
# Corrección
lower = max(0, Q1 - 1.5*IQR)  # el mínimo lógico es 0
print(f"Límite inferior: {lower:.2f}")

In [ ]:
# Reemplaza valores 
df["fare"] = df["fare"].clip(lower, upper)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12,5))

ax[0].boxplot(df["fare"], vert=False)
ax[0].set_title("Fare - Después (boxplot)")

ax[1].hist(df["fare"], bins=30, color="lightgreen", edgecolor="black")
ax[1].set_title("Fare - Después (histograma)")

plt.show()

**OJO: La forma de manejar los _outliers_ depende del contexto y del análisis que se vaya a realizar. No existe una única solución correcta.**

## Sigamos con el preprocesamiento de datos

El preprocesamiento de datos **no es un conjunto de pasos fijos**, sino que depende de:

1. **El algoritmo de Machine Learning que vamos a usar**  
   - Algunos algoritmos **son sensibles a la escala** (ej: regresión logística, SVM, KNN, redes neuronales).  
     → Necesitan escalado o normalización.  
   - Otros algoritmos **no lo necesitan** (ej: árboles de decisión, random forest, gradient boosting).  
     → Pueden trabajar con variables en distintas escalas sin problema.

2. **El tipo de problema**  
   - Si tengo muchas variables categóricas con muchas categorías → One-Hot Encoding puede generar demasiadas columnas, mejor usar *target encoding* o *embeddings*.  
   - Si el dataset tiene muchas variables correlacionadas → PCA u otra técnica de reducción de dimensionalidad puede ser útil.  

3. **El objetivo final**  
   - Predecir con máxima precisión.  
   - Hacer un modelo interpretable.  
   - Reducir tiempo de cómputo.  

En este sentido, y para que el ejemplo sea más real, supongamos que nos plantemos lo siguiente:

_Podemos predicir si un pasajero sobrevivió o no al naufragio del Titanic, usando características (edad, sexo, clase,etc) como variables predictoras?_

Aquí estamos hablando de un **problema de clasificación.**

* Variable objetivo: `survived` (0 = no sobrevivió, 1 = sobrevivió)
* Variables predictoras: `pclass`, `sex`, `age`, `sibsp`, `parch`, `fare`, `adult_male`.

---

#### Técnicas de preprocesamiento que tienen sentido en este caso

1. Valores categóricos: `sex`
    * Necesitans ser convertidos a variables numéricas.
    * Técnicas: One-Hot Encoding, Label Encoding, Target Encoding.

2. Escalado de variables numéricas: `age`, `fare`
    * Si usamos modelos como _Regresión logística_ o _SVM_, el escalado es importante porque dependen de las distancias y magnitudes.
    * Si usamos _árboles de decisión_ o _random forest_, el escalado no es tan crítico.
    * Técnicas: Min-Max Scaling, Standardization (Z-score), Robust Scaling.

3. Separación en _train/test_
    * Es fundamental para evaluar el rendimiento del modelo en datos no vistos.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# Definir X e y
X = df[["pclass", "sex", "age", "sibsp", "parch", "fare", "adult_male"]]
y = df["survived"]

Separamos los datos en conjuntos de entrenamiento (80%) y prueba (20%) para evaluar el modelo de manera justa.

In [ ]:
# Separar train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
# Definir columnas numéricas y categóricas
num_cols = ["age", "sibsp", "parch", "fare"]
cat_cols = ["sex", "pclass"]
bin_cols = ["adult_male"] 

   - **Numéricas** → `StandardScaler` para escalar a media 0 y varianza 1.
   - **Categóricas** → `OneHotEncoder(drop="first")` para convertir categorías en variables dummy.
   - **Binaria (`adult_male`)** → `passthrough`, se deja igual porque ya está en formato 0/1.

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(drop="first"), cat_cols),
        ("bin", "passthrough", bin_cols)
    ]
)

Se construyó un `Pipeline` que aplica el preprocesamiento definido y luego entrena una **Regresión Logística**.

In [ ]:
#Pipeline con regresión logística
log_reg = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(max_iter=10000))
])

El modelo se entrenó con los datos de entrenamiento.

In [ ]:
# Entrenar modelo
log_reg.fit(X_train, y_train)

Se evaluó en los datos de prueba usando **accuracy** y métricas de clasificación (precisión, recall, F1).

In [ ]:
# Evaluación
y_pred = log_reg.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))